In [1]:
import pandas as pd
from ast import literal_eval
import numpy as np
import torch
import models.recipe_encoder as recipe

%load_ext autoreload
%autoreload 2

csv_file = 'archive/updated_data_with_lists.csv'
df = pd.read_csv(csv_file, converters={"Cleaned_Ingredients": literal_eval, "Instructions": literal_eval})
print(len(df))
df.head()

13496


,Unnamed: 0,Title,Instructions,Image_Name,Cleaned_Ingredients
0,0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"[Pat chicken dry with paper towels, season all...",miso-butter-roast-chicken-acorn-squash-panzanella,"[1 (3½–4-lb.) whole chicken, 2¾ tsp. kosher sa..."
1,1,Crispy Salt and Pepper Potatoes,[Preheat oven to 400°F and line a rimmed bakin...,crispy-salt-and-pepper-potatoes-dan-kluger,"[2 large egg whites, 1 pound new potatoes (abo..."
2,2,Thanksgiving Mac and Cheese,[Place a rack in middle of oven; preheat to 40...,thanksgiving-mac-and-cheese-erick-williams,"[1 cup evaporated milk, 1 cup whole milk, 1 ts..."
3,3,Italian Sausage and Bread Stuffing,[Preheat oven to 350°F with rack in middle. Ge...,italian-sausage-and-bread-stuffing-240559,"[1 (¾- to 1-pound) round Italian loaf, cut int..."
4,4,Newton's Law,[Stir together brown sugar and hot water in a ...,newtons-law-apple-bourbon-cocktail,"[1 teaspoon dark brown sugar, 1 teaspoon hot w..."


In [2]:
column = df["Cleaned_Ingredients"]
# item_list = [word
#                     for w_list in column
#                     for str_list in w_list
#                     # Make period and comma separate words
#                     # # Remove parentheses and quotes
#                     for word in str_list.replace('.', ' . ').replace(',',' , ').replace('(','').replace(')','').replace('"','').split()]

item_list = []
element_list = []
ingr_max = 0
print("Column length", len(column))
for w_list in column:
    tmp =[]
    for str_list in w_list:
        # for word in str_list:
        formatted = str_list.replace('.', ' . ').replace(',',' , ').replace('(',' ( ').replace(')',' ) ').replace('"','').split()
        item_list.extend(formatted)
        new_len = len(formatted)
        tmp.extend(formatted)
        if new_len>ingr_max:
            ingr_max=new_len
    element_list.append(tmp)

print(item_list[:10])
print(element_list[0])
print("Len Elements:", len(element_list))
ingr_vocab = set(item_list)
ingr_vocab_size = len(ingr_vocab)
print("Size:", ingr_vocab_size)
print("Max:", ingr_max)

Column length 13496
['1', '(', '3½–4-lb', '.', ')', 'whole', 'chicken', '2¾', 'tsp', '.']
['1', '(', '3½–4-lb', '.', ')', 'whole', 'chicken', '2¾', 'tsp', '.', 'kosher', 'salt', ',', 'divided', ',', 'plus', 'more', '2', 'small', 'acorn', 'squash', '(', 'about', '3', 'lb', '.', 'total', ')', '2', 'Tbsp', '.', 'finely', 'chopped', 'sage', '1', 'Tbsp', '.', 'finely', 'chopped', 'rosemary', '6', 'Tbsp', '.', 'unsalted', 'butter', ',', 'melted', ',', 'plus', '3', 'Tbsp', '.', 'room', 'temperature', '¼', 'tsp', '.', 'ground', 'allspice', 'Pinch', 'of', 'crushed', 'red', 'pepper', 'flakes', 'Freshly', 'ground', 'black', 'pepper', '⅓', 'loaf', 'good-quality', 'sturdy', 'white', 'bread', ',', 'torn', 'into', '1', 'pieces', '(', 'about', '2½', 'cups', ')', '2', 'medium', 'apples', '(', 'such', 'as', 'Gala', 'or', 'Pink', 'Lady;', 'about', '14', 'oz', '.', 'total', ')', ',', 'cored', ',', 'cut', 'into', '1', 'pieces', '2', 'Tbsp', '.', 'extra-virgin', 'olive', 'oil', '½', 'small', 'red', 'onion',

In [3]:
df["Cleaned_Ingredients"][0]

['1 (3½–4-lb.) whole chicken',
 '2¾ tsp. kosher salt, divided, plus more',
 '2 small acorn squash (about 3 lb. total)',
 '2 Tbsp. finely chopped sage',
 '1 Tbsp. finely chopped rosemary',
 '6 Tbsp. unsalted butter, melted, plus 3 Tbsp. room temperature',
 '¼ tsp. ground allspice',
 'Pinch of crushed red pepper flakes',
 'Freshly ground black pepper',
 '⅓ loaf good-quality sturdy white bread, torn into 1" pieces (about 2½ cups)',
 '2 medium apples (such as Gala or Pink Lady; about 14 oz. total), cored, cut into 1" pieces',
 '2 Tbsp. extra-virgin olive oil',
 '½ small red onion, thinly sliced',
 '3 Tbsp. apple cider vinegar',
 '1 Tbsp. white miso',
 '¼ cup all-purpose flour',
 '2 Tbsp. unsalted butter, room temperature',
 '¼ cup dry white wine',
 '2 cups unsalted chicken broth',
 '2 tsp. white miso',
 'Kosher salt',
 'freshly ground pepper']

In [4]:
column = df["Instructions"]
# item_list = [word
#                     for w_list in column
#                     for str_list in w_list
#                     # Make period and comma separate words
#                     # # Remove parentheses and quotes
#                     for word in str_list.replace('.', ' . ').replace(',',' , ').replace('(','').replace(')','').replace('"','').split()]

item_list = []
inst_max = 0
for w_list in column:
    for str_list in w_list:
        # for word in str_list:
        formatted = str_list.replace('.', ' . ').replace(',',' , ').replace('(','').replace(')','').replace('"','').split()
        item_list.extend(formatted)
        new_len = len(formatted)
        if new_len>inst_max:
            inst_max=new_len
            
print(item_list[:10])
inst_vocab = set(item_list)
inst_vocab_size = len(inst_vocab)
print("Size:",inst_vocab_size)
print("Max:", inst_max)

['Pat', 'chicken', 'dry', 'with', 'paper', 'towels', 'season', 'all', 'over', 'with']
Size: 21164
Max: 247


In [5]:
column = df["Title"].to_list()
# item_list = [word
#                 for str_list in column
#     #                     # Make period and comma separate words
#     #                     # # Remove parentheses and quotes
#                 for word in str_list.replace('.', ' . ').replace(',',' , ').replace('(','').replace(')','').replace('"','').split()
#                 if word]

item_list = []
title_max = 0
for str_list in column:
        # for word in str_list:
        formatted = str_list.replace('.', ' . ').replace(',',' , ').replace('(','').replace(')','').replace('"','').split()
        item_list.extend(formatted)
        new_len = len(formatted)
        if new_len>title_max:
            title_max=new_len

print(item_list[:10])
title_vocab = set(item_list)
title_vocab_size = len(title_vocab)
print("Size:", title_vocab_size)
print("Max:", title_max)

['Miso-Butter', 'Roast', 'Chicken', 'With', 'Acorn', 'Squash', 'Panzanella', 'Crispy', 'Salt', 'and']
Size: 6959
Max: 21


In [6]:
total_vocab = set(list(inst_vocab) + list(ingr_vocab) + list(title_vocab)) 
total_vocab_size = len(total_vocab)
print("Size:",total_vocab_size)
total_max = max([inst_max, ingr_max, title_max])
print("Max:",total_max)

Size: 31551
Max: 247


In [7]:
word_to_ix = {word: i for i, word in enumerate(total_vocab)}

word_to_ix_ingr = {word: i for i, word in enumerate(ingr_vocab)}
word_to_ix_inst = {word: i for i, word in enumerate(inst_vocab)}
word_to_ix_title = {word: i for i, word in enumerate(title_vocab)}

In [8]:
# from torchtext.vocab import vocab

# Get longest sentence in Ingredients or Instructions for max_len
# Use torchtext to make vocab: vocab_size = input_size

In [9]:
def pad_to_value(x, val):
    if x.shape[1]>val:
        raise ValueError(f"{x.shape[1]}>{val}")
    diff = val - x.shape[-1]
    # top, bottom
    pad = (0, diff)
    out = torch.nn.functional.pad(x, pad, mode='constant', value=0)
    return out

In [10]:
# Test recipe encoder:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("You are using device: %s" % device)


## Dummy data between 0 and 10
# Size: batch=2, title_len = 8, ingr_len=15, instr_len=30
src_list = [
            # title
            np.random.rand(2, 8) * 10,
            # ingredients with 5 lines
            np.random.rand(2, 5, 15) * 10,
             # instructions with 7 lines
            np.random.rand(2, 7, 15) * 10
            ]
# Convert to torch and pad to max length
x = [ pad_to_value(torch.tensor(t, dtype=torch.long),total_max) for t in src_list]
print("Input shapes:")
print(x[0].shape)
print(x[1].shape)
print(x[2].shape)




You are using device: cuda
Input shapes:
torch.Size([2, 247])
torch.Size([2, 5, 247])
torch.Size([2, 7, 247])


In [14]:
recipe_encoder = recipe.RecipeEncoder(device="cuda", vocab_size=total_vocab_size,
                                      max_len=total_max)
outputs = recipe_encoder.forward(x)
print(outputs)

(tensor([[[ 0.9685, -0.4398,  0.4960,  ..., -1.0238,  1.2345, -1.2555],
         [ 0.4830,  0.7878,  1.9774,  ...,  0.2230,  0.0140,  0.3211],
         [ 0.3162,  0.7880,  1.3049,  ...,  0.7254, -0.3402,  0.9012],
         ...,
         [ 1.0431, -1.4039, -0.4544,  ..., -0.0405, -0.5345, -1.0366],
         [ 2.2795, -0.8474, -0.9265,  ..., -0.4781,  1.4654, -0.3886],
         [ 1.7715, -0.2514, -1.3991,  ..., -1.1000, -0.3912, -0.1988]],

        [[ 1.0820, -0.0366,  0.5957,  ..., -0.7241,  0.2164, -0.5669],
         [ 0.6265,  0.1062,  2.9680,  ..., -0.3430,  0.3575, -0.2493],
         [ 0.8188,  0.0586,  0.8974,  ...,  0.3198, -0.0324,  0.2937],
         ...,
         [ 0.7138, -1.1093, -0.8000,  ..., -0.2474, -0.3375, -0.7815],
         [ 1.7497, -0.5528, -1.0023,  ..., -0.8555,  1.1080, -1.0275],
         [ 1.4496, -0.5739, -1.2986,  ..., -0.9250, -0.2734, -0.7378]]],
       device='cuda:0', grad_fn=<CatBackward0>), tensor([[-0.7092,  0.2182,  0.0310,  ...,  0.4662, -0.0947, -0.577